# QualiCharge maps

## Introduction

L'utilisation de cartes choroplèthe pour la restitution d'indicateurs QualiCharge nécessite le chargement préalable des contours des zones affichées regroupées par cartes.

Pour des raisons de facilité de consultation, les cartes utilisées sont séparées par zone géographique:

- Territoire européen (régions européennes : 11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93, 94)
- Territoire américain (régions Guadeloupe, Martinique, Guyane : 1, 2, 3)
- Territoire africain (régions La Réunion, Mayotte : 4, 6)

Quatre niveaux de découpage sont proposés:

- régions
- départements
- communes
- agglomérations

Le quatrième niveau est dérivé du découpage communal en intégrant les regroupements de communes définis dans les établissement public de coopération intercommunale (EPCI) à ficalité propre (métropoles, communautés urbaines, communautés d'agglomération, communautés de communes, Aix-Marseille et Grand Paris).

Les régions sont découpées en départements. Les régions ultramarines (hors territoire européen) ne comportent qu'un seul département (code '97x' avec 'x' code région).
Les départements sont découpés en communes.
Les agglomérations sont des regroupements de communes


## Map utilisées

Pour des raisons de lisibilité, les cartes associées à l'ensemble des territoires, au territoire américain et au territoire africain ne sont pas retenues.

Les cartes utilisées pour restituer les indicateurs sont alors les suivantes (la métropole correspond au territoire européen):

- Territoire:
    - métropole par régions
    - métropole par départements
- 18 régions (pour chaque région):
    - région par départements
    - région par ECPI
    - région par commune


## Génération des fichiers

Les fichiers GeoJson pour chaque map sont extraits des [contours administratifs Etalab](https://etalab-datasets.geo.data.gouv.fr/contours-administratifs/2024/geojson/). 
Les régions sont identifiées par leur code [ISO 3166-2:FR](https://fr.wikipedia.org/wiki/ISO_3166-2:FR#R%C3%A9gions_m%C3%A9tropolitaines) :

  - metropole_reg
  - metropole_dep
  - region_xxx_dep
  - region_xxx_ecpi
  - region_xxx_com

  avec xxx : trois caractères région du code ISO 3166-2:FR (ex. ARA pour Auvergne Rhone Alpes)

La précision retenue est de 100 m.

In [3]:
import os
from sqlalchemy import create_engine

engine = create_engine(os.getenv("DATABASE_URL"))

In [35]:
import pandas as pd
query_dep = """SELECT department.code, department.name, ST_AsGeoJSON(department.geometry) :: json -> 'coordinates'  AS polygon, region.code AS reg_code
           FROM department INNER JOIN region ON department.region_id = region.id 
           WHERE region.name = 'Auvergne-Rhône-Alpes'"""
with engine.connect() as conn:
    df = pd.read_sql_query(query_dep, conn)

maps_dir = './maps/'
reg = '84'
file_dep = 'region_' + reg + '_dep.json'
contour = {'type': 'FeatureCollection', 
           'features': [{'type': 'Feature', 
                         'properties': {'code': row[1], 'nom': row[2]}, 
                         'geometry': {'type': 'Polygon', 'coordinates': row[3]}} 
                        for row in list(df.itertuples()) if row[4] == reg]}
with open(maps_dir + file_dep, 'w', encoding ='utf8') as map_file:
    json.dump(contour, map_file)
print(contour['features'][0])
# {'properties': {'code': '01', 'nom': 'Ain'}}

{'type': 'Feature', 'properties': {'code': '01', 'nom': 'Ain'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[5.8251, 45.9386], [5.8292, 45.9383], [5.8263, 45.9361], [5.8227, 45.9307], [5.8226, 45.927], [5.8284, 45.9207], [5.8297, 45.9162], [5.8291, 45.9131], [5.8262, 45.9038], [5.82, 45.8885], [5.8152, 45.8772], [5.8088, 45.8625], [5.8075, 45.8606], [5.8034, 45.8552], [5.801, 45.8457], [5.7972, 45.8311], [5.7953, 45.83], [5.7889, 45.8267], [5.7878, 45.8235], [5.7849, 45.8182], [5.7844, 45.8138], [5.787, 45.8095], [5.7869, 45.8058], [5.7849, 45.7972], [5.7842, 45.7897], [5.783, 45.7836], [5.7811, 45.781], [5.7804, 45.7773], [5.7834, 45.7737], [5.7834, 45.7724], [5.7826, 45.7679], [5.7793, 45.7618], [5.7808, 45.7568], [5.7835, 45.754], [5.784, 45.752], [5.7836, 45.745], [5.7805, 45.7427], [5.7735, 45.7425], [5.7698, 45.7408], [5.7715, 45.737], [5.7766, 45.7292], [5.7771, 45.7272], [5.7747, 45.7241], [5.7662, 45.7212], [5.7631, 45.7186], [5.7601, 45.7146], [5.7583, 45.7108], [5.7556,

In [ ]:
data = requests.get(source_dir + 'epci-100m.geojson')
jsn = json.loads(data.content)
print([feature['properties'] for feature in jsn['features']])

In [36]:
import json 
import requests

reg_metro = ['11', '24', '27', '28', '32', '44', '52', '53', '75', '76', '84', '93', '94']
dpt_metro = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29',
             '2A', '2B', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
             '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', 
             '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95']
# ara_dep = 
source_dir = 'https://etalab-datasets.geo.data.gouv.fr/contours-administratifs/2024/geojson/'
maps_dir = './maps/'

# une seule map pour l'instant -> à compléter
maps = [ ['regions-100m.geojson', reg_metro, 'metropole_reg.json'],
         ['departements-100m.geojson', dpt_metro, 'metropole_dep.json']
       ]
for map in maps[1:]:
    data = requests.get(source_dir + map[0])
    jsn = json.loads(data.content)
    # print([feature['properties']['code'] for feature in jsn['features']])
    print(jsn['features'][0])
    contour = {'type': 'FeatureCollection', 'features': [feature for feature in jsn['features'] if feature['properties']['code'] in map[1]]}
    #print([feature['properties']['code'] for feature in contour['features']])
    with open(maps_dir + map[2], 'w', encoding ='utf8') as map_file:
        json.dump(contour, map_file)


{'type': 'Feature', 'properties': {'code': '01', 'nom': 'Ain', 'region': '84'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[5.8251, 45.9386], [5.8292, 45.9383], [5.8263, 45.9361], [5.8227, 45.9307], [5.8226, 45.927], [5.8284, 45.9207], [5.8297, 45.9162], [5.8291, 45.9131], [5.8262, 45.9038], [5.82, 45.8885], [5.8152, 45.8772], [5.8088, 45.8625], [5.8075, 45.8606], [5.8034, 45.8552], [5.801, 45.8457], [5.7972, 45.8311], [5.7953, 45.83], [5.7889, 45.8267], [5.7878, 45.8235], [5.7849, 45.8182], [5.7844, 45.8138], [5.787, 45.8095], [5.7869, 45.8058], [5.7849, 45.7972], [5.7842, 45.7897], [5.783, 45.7836], [5.7811, 45.781], [5.7804, 45.7773], [5.7834, 45.7737], [5.7834, 45.7724], [5.7826, 45.7679], [5.7793, 45.7618], [5.7808, 45.7568], [5.7835, 45.754], [5.784, 45.752], [5.7836, 45.745], [5.7805, 45.7427], [5.7735, 45.7425], [5.7698, 45.7408], [5.7715, 45.737], [5.7766, 45.7292], [5.7771, 45.7272], [5.7747, 45.7241], [5.7662, 45.7212], [5.7631, 45.7186], [5.7601, 45.7146], [5.7583, 45

## Utilisation des fichiers

Dans le menu `Admin settings / Maps / add a map` de Metabase, les informations suivantes sont à documenter:

- name (nom de la map),
- URL du fichier (geojson),
- id (lien avec les données à représenter - property geojson),
- name (nom affiché - property geojson)

exemple : https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions-version-simplifiee.geojson (property_id = code_region)